#### Part that can be used for extractions. Tell the llm to write the query. Use the sql cmd generated from the llm to query the db to get an extraction (table) not only a single answer.

In [5]:
from langchain_ollama import OllamaLLM
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [6]:

llm = OllamaLLM(model="sqlcoder", temperature=0)   # best model at the moment defog_sqlcoder-7b-2


In [7]:
db_user = "root"
db_password = "1234"
db_host = "localhost"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(
    f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",
    sample_rows_in_table_info=3
)

In [8]:
db_chain1 = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True)

response = db_chain1.invoke("Extract all rows where brand is Levi")
response["intermediate_steps"]

def extract_sql_cmd(intermediate_steps):
    for step in intermediate_steps:
        # Case 1: dictionary with key 'sql_cmd'
        if isinstance(step, dict) and 'sql_cmd' in step:
            return step['sql_cmd']
        # Case 2: string that looks like an SQL query
        if isinstance(step, str) and step.strip().lower().startswith("select"):
            return step
    return None  # fallback if not found

# Example usage
sql_cmd = extract_sql_cmd(response["intermediate_steps"])
print("SQL Query:", sql_cmd)


"""def extract_sql_result(intermediate_steps):
    for step in intermediate_steps:
        if isinstance(step, str) and step.startswith("[("):  # likely a result set
            return step
    return None

sql_result = extract_sql_result(response["intermediate_steps"])
print("SQL Result:", sql_result)"""


import pandas as pd
from sqlalchemy import create_engine
engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
df = pd.read_sql(sql_cmd, engine)

# --- Step 5: Save to Excel ---
output_file = "levi_tshirts.xlsx"
df.to_excel(output_file, index=False)

print(f"✅ Excel file saved as: {output_file}")





> Entering new SQLDatabaseChain chain...
Extract all rows where brand is Levi
SQLQuery:SELECT * FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(29, 'Levi', 'Red', 'XS', 45, 74), (51, 'Levi', 'Red', 'M', 31, 98), (11, 'Levi', 'Red', 'XL', 16, 96), (37, 'Levi', 'Blue', 'XS', 10, 49), (32, 'Levi', 'Blue', 'M', 21, 24), (90, 'Levi', 'Blue', 'XL', 25, 33), (36, 'Levi', 'Black', 'S', 49, 71), (88, 'Levi', 'Black', 'M', 14, 95), (12, 'Levi', 'Black', 'L', 47, 61), (27, 'Levi', 'White', 'XS', 34, 72), (31, 'Levi', 'White', 'S', 23, 51), (78, 'Levi', 'White', 'M', 21, 85), (57, 'Levi', 'White', 'L', 47, 60)]
Answer:Levi shirts are available in all sizes.
> Finished chain.
SQL Query: SELECT * FROM t_shirts WHERE brand = 'Levi'


ModuleNotFoundError: No module named 'openpyxl'